## Setup

In [1]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib qt
# matplotlib.backend
# matplotlib.validate_backend("GTK3Ag")

In [2]:
import numpy as np
from tqdm.auto import tqdm as progressbar
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
import cv2.aruco as aruco
import common as c
from os.path import join
from calib import calibrate_charuco_local, load_board, load_coefficients, save_coefficients

## Funcs

In [4]:
arucoParams = aruco.DetectorParameters_create()

def find_charuco_corners(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    corners, ids, rejected = aruco.detectMarkers( gray, aruco_dict, cameraMatrix=mtx, distCoeff=dist, parameters=arucoParams)
    resp, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco( markerCorners=corners, markerIds=ids, image=gray, board=board, cameraMatrix=mtx, distCoeffs=dist)
    return charuco_corners

def find_charuco_pose(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    corners, ids, rejected = aruco.detectMarkers( gray, aruco_dict, cameraMatrix=mtx, distCoeff=dist, parameters=arucoParams)
    resp, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco( markerCorners=corners, markerIds=ids, image=gray, board=board, cameraMatrix=mtx, distCoeffs=dist)

    retval, rvec, tvec	=	cv2.aruco.estimatePoseCharucoBoard(charuco_corners, charuco_ids, board, mtx, dist, None,None)
    return rvec, tvec

def mask_rgb(img, mask):
	ret = np.zeros_like(img)
	for i in range(ret.shape[-1]):
		ret[:,:,i] = img[:,:,i] * mask
	return ret

def get_charuco_rect(board):
	delim = np.max(board.chessboardCorners, axis=0)
	b =board.getSquareLength() 
	corner_points = [
		[b, b, 0],
		[delim[0], b, 0],
		delim,
		[b, delim[1], 0],
	]
	return np.array(corner_points)

def get_charuco_mask(image, board, rvec, tvec, mtx, dist):
	if image.ndim==2:
		mask = np.zeros_like(image)
	elif image.ndim==3:
		mask = np.zeros(image.shape[:2])

	rect = get_charuco_rect(board)
	points, _ = cv2.projectPoints(rect, rvec, tvec, mtx, dist)
	proj_points_round = np.round(points, 0).astype(np.int32)
	cv2.fillPoly(mask, [proj_points_round], True, 255 )
	return mask

def crop_charuco_board(image, board, rvec, tvec, mtx, dist):
	mask = get_charuco_mask(image, board, rvec, tvec, mtx, dist)
	if image.ndim == 3:
		return mask_rgb(image, mask)
	elif image.ndim==2:
		return mask*image

def chess_plane(rvec, tvec):
	# Find p
	n_chessboard = np.array([0, 0, 1])
	M,_ = cv2.Rodrigues(rvec)
	n3 = M@n_chessboard
	p = -1 * n3@tvec
	n = np.hstack((n3,p))
	return n

In [15]:
mtx, dist = load_coefficients("/home/freitas/TCC/ender-laser-scanner/calib_data/calibration_charuco.yml")

In [18]:
def find_charuco_pose(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    corners, ids, rejected = aruco.detectMarkers( gray, aruco_dict, cameraMatrix=mtx, parameters=arucoParams)
    resp, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco( markerCorners=corners, markerIds=ids, image=gray, board=board, cameraMatrix=mtx)

    if(resp > 6):
        retval, rvec, tvec	=	cv2.aruco.estimatePoseCharucoBoard(charuco_corners, charuco_ids, board, mtx, dist, None,None)
    else:
        return None, None
    if not retval:
        print("erro")
    return rvec, tvec

def find_charuco_pose_c(image, mtx, dist):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    corners, ids, rejected = aruco.detectMarkers( gray, aruco_dict, cameraMatrix=mtx, parameters=arucoParams, distCoeff=dist)
    resp, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco( markerCorners=corners, markerIds=ids, image=gray, board=board, cameraMatrix=mtx, distCoeffs=dist)

    if(resp > 6):
        retval, rvec, tvec	=	cv2.aruco.estimatePoseCharucoBoard(charuco_corners, charuco_ids, board, mtx, dist, None,None)
    else:
        return None, None
    if not retval:
        print("erro")
    return rvec, tvec

In [7]:
data_path="/home/freitas/TCC/ender-laser-scanner/pics/v1_raspi/auto_exp"

In [8]:
import json

if("8x8" in data_path):
    board, aruco_dict = load_board("board_8x8.pkl")
else:
    board, aruco_dict = load_board("board.pkl")

print(board.getChessboardSize())
print(board.getMarkerLength())
print(board.getSquareLength())

data = {}
images = []
rvecs = []
tvecs = []
with open(join(data_path,"points.json")) as f:
    data = json.loads(f.read())

for fig, point in data.items():
    image = cv2.cvtColor(cv2.imread(join(data_path, fig)), cv2.COLOR_BGR2RGB)
    images.append((point, image))
    rvec, tvec = find_charuco_pose(image)
    if rvec is None or tvec is None:
        print("OOPS")
    rvecs.append(rvec)
    tvecs.append(tvec)

(16, 16)
9.375
12.5


In [12]:
plt.imshow(images[0][1])
plt.show()

In [11]:
P = np.stack((i[0] for i in images))

/home/freitas/TCC/ender-laser-scanner/tccenv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3361: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


In [12]:
c.animate((g[1] for g in images))

## Calib

In [12]:
idxs, mtx_re, dist_re, rvecs, tvecs, corners = calibrate_charuco_local((d[1] for d in images), board, aruco_dict, prior=(mtx, dist))

# discard boards with not enough points
# images = [ images[i] for i in idxs ]

reprojection error: 1.8591463075536554


In [16]:
print(mtx)
print(mtx_re)
print("===========")
print(dist)
print(dist_re)

[[2.59128920e+03 0.00000000e+00 1.62581996e+03]
 [0.00000000e+00 2.59961593e+03 1.31712834e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[2.84415758e+03 0.00000000e+00 1.63183539e+03]
 [0.00000000e+00 2.96575308e+03 1.29320111e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[ 0.18142624 -0.39657441  0.0105025  -0.0015666   0.10279771]]
[[ 0.24748201 -0.68514103  0.00086784 -0.00512334  0.39862164]]


In [7]:
for i, image in enumerate(images):
    images[i] = image[0], cv2.undistort(image[1], mtx, dist, None, mtx)

dist = None

NameError: name 'mtx' is not defined

### Calib with prior

In [ ]:
# ( Problem: maybe this undistorts twice? Don't use before checking)
# plot = False
# idxs, mtx, dist, rvecs, tvecs, corners = calibrate_charuco_local((d[1] for d in images), board, aruco_dict, prior=(mtx_manual_calib,dist_manual), plot=plot) 
# # Once again remove images with too little markers
# images = [images[i] for i in idxs ]

# for i, image in enumerate(images):
#     images[i] = image[0], cv2.undistort(image[1], mtx, dist, None, mtx)

# Laser

## Inspecionar Laser

In [15]:
def find_board_and_laser(image):
    """Crop out the image to contain only charuco board, and find the laser points in this image
    @param image original image
    @param rvec rotation vec of the board
    @param tvec translation vec of the board
    @returns: (masked, centroids), tuple containing the masked image with only the board, and vector of laser points """
    rvec, tvec = find_charuco_pose(image)
    masked = crop_charuco_board(image, board, rvec, tvec, mtx, dist)
    print(masked.shape)
    centroids = c.column_centroids(c.red_contrast(masked), mask=c.red_contrast(masked)>10)
    try:
        p_centroids = np.stack(list((i, p, 1) for i,p in enumerate(centroids) if not np.isnan(p))).T
    except ValueError:
        p_centroids = np.array([[],[]])
    return masked, p_centroids

### Plot de todas as linhas

In [114]:
n=0
for point, image in images:
    masked, p_centroids = find_board_and_laser(image)
    plt.plot(p_centroids[1], label=n)
    n+=1
plt.legend() 

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)


### Inspecionar linha específica

### Animar Laser

In [115]:
from matplotlib import animation

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
# animation function.  This is called sequentially
imgplot = None
pointsplot = None

def init():
    global imgplot, pointsplot
    imgplot = plt.imshow(images[0][1], animated=True)
    pointsplot, = plt.plot(range(images[0][1].shape[0]), "r", animated=True)
    return pointsplot, imgplot

def laser_points_img_gen(i):
    global imgplot, pointsplot
    # Find board and laser
    point, image = images[i]
    masked, centroids = find_board_and_laser(image)
    print(masked.shape)

    # Update anim
    imgplot.set_data(masked)
    pointsplot.set_data(centroids[0], centroids[1])
    return pointsplot, imgplot

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, laser_points_img_gen, init_func=init,
                               frames=len(images), interval=500, blit=True)

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280

## Animar pontos

In [20]:
# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
# animation function.  This is called sequentially
imgplot = None
pointsplot = None
chessplot = None
chessplot_reproj = None


def init():
    global imgplot, pointsplot, chessplot, chessplot_reproj
    imgplot = plt.imshow(images[0][1], animated=True)
    pointsplot, = plt.plot(np.zeros(images[0][1].shape[0]), "r", animated=True)
    chessplot, = plt.plot(range(images[0][1].shape[0]), "b.", animated=True)
    chessplot_reproj, = plt.plot(range(images[0][1].shape[0]), "g.", animated=True)
    return pointsplot, imgplot, chessplot, chessplot_reproj

def laser_points_img_gen(i):
    global imgplot, pointsplot, chessplot, chessplot_reproj
    # Find board and laser
    point, image  = images[i]
    masked, centroids = find_board_and_laser(image)
    charuco_corners = find_charuco_corners(image)
    rvec, tvec = find_charuco_pose(image)
    proj_points, jacobian = cv2.projectPoints(board.chessboardCorners, rvec, tvec, mtx, dist)

    # Update anim
    imgplot.set_data(image)
    # pointsplot.set_data(centroids[0], centroids[1])
    chessplot.set_data(charuco_corners[..., 0], charuco_corners[..., 1])
    chessplot_reproj.set_data(proj_points[..., 0], proj_points[..., 1])
    return pointsplot, imgplot, chessplot, chessplot_reproj

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, laser_points_img_gen, init_func=init,
                               frames=200, interval=500, blit=True)

(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)


Traceback (most recent call last):
  File "/home/freitas/TCC/ender-laser-scanner/tccenv/lib/python3.9/site-packages/matplotlib/backend_bases.py", line 1216, in _on_timer
    ret = func(*args, **kwargs)
  File "/home/freitas/TCC/ender-laser-scanner/tccenv/lib/python3.9/site-packages/matplotlib/animation.py", line 1477, in _step
    still_going = super()._step(*args)
  File "/home/freitas/TCC/ender-laser-scanner/tccenv/lib/python3.9/site-packages/matplotlib/animation.py", line 1189, in _step
    self._draw_next_frame(framedata, self._blit)
  File "/home/freitas/TCC/ender-laser-scanner/tccenv/lib/python3.9/site-packages/matplotlib/animation.py", line 1208, in _draw_next_frame
    self._draw_frame(framedata)
  File "/home/freitas/TCC/ender-laser-scanner/tccenv/lib/python3.9/site-packages/matplotlib/animation.py", line 1776, in _draw_frame
    self._drawn_artists = self._func(framedata, *self._args)
  File "/tmp/ipykernel_49815/4070411308.py", line 21, in laser_points_img_gen
    point, ima

In [19]:
i = 19
(point, image), rvec, tvec = images[i], rvecs[i], tvecs[i]
# masked, p_centroids = find_board_and_laser(image, rvec, tvec)
charuco_corners=find_charuco_corners(image)

rvec, tvec = find_charuco_pose_c(image, mtx, dist)
proj_points, jacobian = cv2.projectPoints(board.chessboardCorners, rvec, tvec, mtx, dist)

plt.imshow(image)
# plt.plot(p_centroids[0],p_centroids[1])
plt.plot(charuco_corners[..., 0],charuco_corners[..., 1], "b.")
plt.plot(proj_points[..., 0],proj_points[..., 1], "g.")

rvec, tvec = find_charuco_pose_c(image, mtx_re, dist_re)
proj_points, jacobian = cv2.projectPoints(board.chessboardCorners, rvec, tvec, mtx_re, dist_re)
plt.plot(proj_points[..., 0],proj_points[..., 1], "r.")

## Plotar Eixos coords

In [27]:
def set_axes_equal(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    '''

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])

In [28]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

zs = []
t0 = None
points = pd.DataFrame(columns=["c1","c2","c3","space","p1","p2","p3"])
for i, (rvec, tvec) in enumerate(zip(rvecs, tvecs)):
    R, jac = cv2.Rodrigues(rvec)
    inv = np.zeros((4,4))
    inv[:3, :3] = R
    inv[:3, 3] = tvec.T
    inv[3,3] = 1

    inv = np.linalg.inv(inv)

    tvec = -R.T@tvec
    if t0 is None:
        t0 = tvec
    R = inv[:3,:3]


    cam = (tvec.T - t0.T).flatten()
    p = images[i][0]

    data = dict(c1=cam[0],c2=cam[1],c3=cam[2], p1=p[0],p2=p[1],p3=p[2])
    points = points.append(data, ignore_index=True)

    # X
    R*=10
    ax.quiver(tvec[0],tvec[1], tvec[2], R[0,0], R[1,0], R[2,0], color="r")
    # Y
    ax.quiver(tvec[0],tvec[1], tvec[2], R[0,1], R[1,1], R[2,1], color="g")
    # Z
    ax.quiver(tvec[0],tvec[1], tvec[2], R[0,2], R[1,2], R[2,2], color="b")
    zs.append(tvec[-1])

# Chessboard
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.scatter(board.chessboardCorners[:,0], board.chessboardCorners[:,1], board.chessboardCorners[:,2], color="k", label="board")
set_axes_equal(ax)
plt.legend()

In [24]:
points["p1"] -= 70.0
points["p2"] -= 50.0
points["p3"] -= 100.0

In [25]:
points["space"] = ""
points.round(1).head(n=20)

,c1,c2,c3,space,p1,p2,p3
0,0.0,0.0,0.0,,0.0,90.0,0.0
1,-0.1,-54.4,-0.7,,55.0,90.0,0.0
2,-95.4,0.2,20.3,,0.0,0.0,20.0
3,-95.7,-54.0,19.7,,55.0,0.0,20.0
4,-51.4,-108.0,19.6,,110.0,45.0,20.0
5,-51.2,-54.2,19.0,,55.0,45.0,20.0
6,-50.9,0.2,19.8,,0.0,45.0,20.0
7,-4.6,-0.1,20.5,,0.0,90.0,20.0
8,-4.7,-54.4,20.0,,55.0,90.0,20.0
9,-4.4,-108.3,20.5,,110.0,90.0,20.0


In [27]:
plt.scatter(-points["c1"], -points["c2"], label="camera")
plt.scatter(points["p1"], points["p2"], label="point")
plt.legend()
# plt.scatter(points["c1"], points["c2"])

In [ ]:
import pandas as pd

print(pd.Series(zs).describe())
print(max(zs) - min(zs))

In [ ]:
(R/10)[:,2]

## Calibrar

In [119]:
def ray_plane_intersect_vectorized(img_points, plane, cam_mtx):
	"""
	Calculate 3d-intersection between image rays defined by `img_ponts` and `cam_mtx`
	and a plane defined by the 4-vec `plane`
	@param img_points np.array with shape (3, N), N= number of points, homegeneous coords
	@param plane np.array with shape = (4,)
	@param cam_mtx np.array with shape = (3,3), intrinsic camera parameters
	@returns np.array with shape (4, N) with 3d points
	"""
	assert plane.shape == (4,)
	assert len(img_points.shape) == 2
	assert img_points.shape[0] == 3

	rays = np.linalg.inv(cam_mtx)@img_points
	p = plane[-1]
	n = plane[:3]
	out = (-p/(n@rays)) * rays
	out = np.vstack((out, np.ones((1, out.shape[-1])))) # homogeneous coords
	return out

### Achar todos os pontos 3D

In [129]:
from collections import defaultdict
Xlist = []

zcounter = defaultdict(int)
for point, image in images:
	# count z 
	zcounter[point[-1]] += 1

	masked, p_centroids = find_board_and_laser(image)
	rvec, tvec = find_charuco_pose(image)
	n = chess_plane(rvec, tvec)
	print(point[-1])
	print(tvec)
	print(n)
	print("=====")
	points3d = ray_plane_intersect_vectorized(p_centroids, n, mtx)
	Xlist.append(points3d)
print(zcounter)

(720, 1280, 3)
100.0
[[144.67776372]
 [111.62973005]
 [112.15128966]]
[-5.75312728e-02 -4.48971365e-01 -8.91692136e-01  1.58446471e+02]
=====
(720, 1280, 3)
100.0
[[ 89.86715147]
 [115.01446189]
 [113.32200053]]
[-5.46218044e-02 -4.52179511e-01 -8.90252856e-01  1.57801124e+02]
=====
(720, 1280, 3)
120.0
[[142.16035982]
 [ 38.81563255]
 [172.96711339]]
[-5.57605489e-02 -4.65312852e-01 -8.83388199e-01  1.78785459e+02]
=====
(720, 1280, 3)
120.0
[[ 87.42443225]
 [ 42.27712763]
 [174.36208374]]
[-5.25357430e-02 -4.68565332e-01 -8.81865367e-01  1.78166387e+02]
=====
(720, 1280, 3)
120.0
[[ 32.89546487]
 [ 85.33421018]
 [154.16837435]]
[-7.62390285e-02 -4.69146690e-01 -8.79823274e-01  1.78183104e+02]
=====
(720, 1280, 3)
120.0
[[ 89.19277007]
 [ 81.88462039]
 [152.36841022]]
[-5.39933963e-02 -4.72346531e-01 -8.79757619e-01  1.77541007e+02]
=====
(720, 1280, 3)
120.0
[[144.00338258]
 [ 78.541086  ]
 [151.13057416]]
[-5.69087566e-02 -4.68919611e-01 -8.81405578e-01  1.78231840e+02]
=====
(720, 

In [130]:
X = np.hstack(Xlist)

### Mostrar pontos 3D

In [131]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[0],X[1], X[2], color="r")

### Fit num plano

In [132]:
u, s, vt = np.linalg.svd(X.T, full_matrices=False)
v = vt.T
n = v[:,-1]
print(n)

[-5.56670943e-04 -1.34467962e-02  6.27881954e-03 -9.99889719e-01]


### Plot Plano e pontos

In [133]:
x0, xf = np.min(X[0]), np.max(X[0])
y0, yf = np.min(X[1]), np.max(X[1])
xplot = np.linspace(x0,xf)
yplot = np.linspace(y0,yf)
xplot, yplot = np.meshgrid(xplot, yplot)
a,b,c,d = n
zplot = -(a*xplot + b*yplot + d)/c

# Resultado

## Plot

In [134]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[0],X[1], X[2], color="r")
ax.plot_surface(xplot, yplot, zplot, color="b", alpha=0.5)

## Params

In [135]:
print("=== Intrinsic Camera Params ===")
print("===== Camera Matrix =====")
print(mtx)
print("===== Lens Distortion Params =====")
print(dist.T)
print("=== Laser Params ===")
print("==== Laser Plane 4-vec ====")
print(n)
print("==== Laser Plane Equation ====")
print(f"{1e3*a:.2f}x + {1e3*b:.2f}y + {1e3*c:.2f}z + {1e3*d:.2f} = 0")

=== Intrinsic Camera Params ===
===== Camera Matrix =====
[[1.61434793e+03 0.00000000e+00 6.92329652e+02]
 [0.00000000e+00 1.63431381e+03 3.69594199e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
===== Lens Distortion Params =====
[[ 3.17169766e-01]
 [-4.13122784e+00]
 [ 3.50115209e-03]
 [ 3.84789825e-03]
 [ 2.27792728e+01]]
=== Laser Params ===
==== Laser Plane 4-vec ====
[-5.56670943e-04 -1.34467962e-02  6.27881954e-03 -9.99889719e-01]
==== Laser Plane Equation ====
-0.56x + -13.45y + 6.28z + -999.89 = 0


## Save

In [137]:
import pickle 

calib = dict(
    mtx = mtx,
    dist= dist,
    n=n
)
with open("calib.pkl", "wb") as f:
    pickle.dump(calib, f)